In [1]:
from matplotlib import pyplot
import pandas as pd
import numpy as np
from datetime import datetime
from geopy.distance import vincenty
from geopy.distance import great_circle
import math

In [2]:
train=pd.read_csv('/home/harsh/dataset/train.csv')
train_fare=train['fare_amount']

# test=pd.read_csv('test.csv')

In [87]:
[nr,nc]=train.shape

temp=train['tip_amount'].mode()
print 'harsh'
print temp
print 'kara'
# train['tip_amount'].fillna(0.0,inplace=True)
# print train.isnull().sum()
train.dtypes



harsh
0    0.0
dtype: float64
kara


TID                           object
vendor_id                     object
new_user                      object
tolls_amount                 float64
tip_amount                   float64
mta_tax                      float64
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
pickup_longitude             float64
pickup_latitude              float64
rate_code                      int64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
surcharge                    float64
fare_amount                  float64
tot_time                     float64
day                            int64
month                          int64
year                           int64
weekday                        int64
hour                           int64
tot_dist                     float64
dtype: object

In [ ]:
a=pd.DataFrame(np.ones((3,1)))
a['cv']=[1,3,2]
a['klp']=a['poi']=np.ones(3)
a

In [12]:
fmt='%Y-%m-%d %H:%M:%S'

train['pickup_datetime']=pd.to_datetime(train['pickup_datetime'])
train['dropoff_datetime']=pd.to_datetime(train['dropoff_datetime'])

train['tot_time']=(((train['dropoff_datetime']-train['pickup_datetime']).astype('timedelta64[s]'))/60).round()
# train['day']= train['pickup_datetime'].dt.day
# train['month'] = train['pickup_datetime'].dt.month
train['year'] = train['pickup_datetime'].dt.year

train['weekday'] = train['pickup_datetime'].dt.dayofweek   

train['hour'] = train['pickup_datetime'].dt.hour
    

In [47]:
#dropping the instances where the time is -ve
a=(train['tot_time']<0)
ind=train['tot_time'][a].index
print len(ind)
print train.shape
train.drop(ind,axis=0,inplace=True)
print train.shape


0
(1658064, 24)
(1658064, 24)


In [58]:
train.loc[[0,1,2,3,4,5]]

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,...,dropoff_latitude,payment_type,surcharge,fare_amount,tot_time,day,month,year,weekday,hour
0,AIX000345001,DST000401,NO,0.0,1.4,0.5,2015-04-20 04:18:25,2015-04-20 04:24:20,1,-74.003939,...,40.734247,CRD,0.5,8.4,6.0,20,4,2015,0,4
1,AIX000345002,DST000401,NO,0.0,1.0,0.5,2015-04-19 18:16:07,2015-04-19 18:22:06,3,-73.973864,...,40.772533,CRD,0.0,8.5,6.0,19,4,2015,6,18
2,AIX000345003,DST000401,NO,0.0,0.0,0.5,2015-04-06 08:04:32,2015-04-06 08:10:05,2,-73.954406,...,40.758350,CSH,0.0,7.0,6.0,6,4,2015,0,8
3,AIX000345004,DST000532,NO,0.0,1.8,0.5,2015-04-10 09:48:00,2015-04-10 10:00:00,2,-73.962345,...,40.756867,CRD,0.0,11.3,12.0,10,4,2015,4,9
4,AIX000345005,DST000401,NO,0.0,0.0,0.5,2015-04-15 13:12:48,2015-04-15 13:26:30,1,-74.004657,...,40.721517,CSH,0.0,10.0,14.0,15,4,2015,2,13
5,AIX000345006,DST000401,NO,0.0,2.4,0.5,2015-04-23 22:04:11,2015-04-23 22:16:11,1,-73.995571,...,40.743319,CRD,NaN,14.4,12.0,23,4,2015,3,22


In [65]:
train.index

Int64Index([      0,       1,       7,       9,      10,      11,      12,
                 13,      14,      15,
            ...
            1658089, 1658090, 1658091, 1658092, 1658093, 1658094, 1658095,
            1658096, 1658097, 1658098],
           dtype='int64', length=1658058)

In [67]:
#train.index is used because in case we drop off some of the rows from dataframe, there can still be accessed using the index
#suppose we delete the row 2,3,4
#then the index of train will have 0,1,5,6,7,...
#but if we access the 2nd row  using the train.loc([2]), then the removed row will be accessed.
# so we should first find the indices which have the data
# however if we use the train.iloc([[2]]) then the 2nd valid row will be accessed.
train['tot_dist'] = [ great_circle( (train['pickup_latitude'][i],train['pickup_longitude'][i]) , (train['dropoff_latitude'][i],train['dropoff_longitude'][i])  ).km   for i in train.index ]

In [85]:
#even if the dist=0 and fare is not zero can be a true case, in case of ROUND TRIP
#(GO SOMEWHERE AND COMES TO SAME PLACE IN SAME CAB)

a=(train['tot_dist']==0)
ind1=train[a].index

a=(train['fare_amount']==0)
ind2=train[a].index

train.loc[ind1]
#see whether to drop these rows or not
# train.drop(ind,axis=0,inplace=True)

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,...,payment_type,surcharge,fare_amount,tot_time,day,month,year,weekday,hour,tot_dist
34,AIX000345035,DST000401,NO,0.00,3.40,0.5,2015-04-23 15:07:00,2015-04-23 15:28:58,1,-74.003282,...,CRD,0.0,20.40,22.0,23,4,2015,3,15,0.0
48,AIX000345049,DST000532,NO,0.00,1.60,0.5,2015-04-11 02:02:00,2015-04-11 02:09:00,3,0.000000,...,CRD,0.5,10.10,7.0,11,4,2015,5,2,0.0
106,AIX000345107,DST000532,NO,0.00,0.00,0.5,2015-04-29 20:12:00,2015-04-29 20:19:00,1,-73.991392,...,CSH,0.5,7.50,7.0,29,4,2015,2,20,0.0
177,AIX000345178,DST000401,NO,0.00,3.20,0.5,2015-04-17 13:14:50,2015-04-17 13:31:50,1,0.000000,...,CRD,0.0,19.20,17.0,17,4,2015,4,13,0.0
190,AIX000345191,DST000401,NO,0.00,0.00,0.5,2015-04-12 15:44:17,2015-04-12 15:47:39,1,0.000000,...,CSH,0.0,5.50,3.0,12,4,2015,6,15,0.0
316,AIX000345317,DST000401,NO,0.00,NaN,0.5,2015-04-12 06:54:01,2015-04-12 07:05:14,1,0.000000,...,CSH,0.0,13.00,11.0,12,4,2015,6,6,0.0
468,AIX000345469,DST000401,NO,0.00,0.00,0.5,2015-04-14 17:37:35,2015-04-14 17:49:54,4,0.000000,...,CSH,1.0,10.50,12.0,14,4,2015,1,17,0.0
474,AIX000345475,DST000401,NO,0.00,2.00,0.5,2015-04-17 01:31:53,2015-04-17 01:36:23,1,0.000000,...,CRD,0.5,9.00,4.0,17,4,2015,4,1,0.0
583,AIX000345584,DST000532,NO,0.00,1.80,0.5,2015-04-27 10:05:00,2015-04-27 10:13:00,1,0.000000,...,CRD,0.0,11.30,8.0,27,4,2015,0,10,0.0
688,AIX000345689,DST000401,NO,0.00,1.90,0.5,2015-04-18 02:14:23,2015-04-18 02:21:24,1,0.000000,...,CRD,0.5,11.40,7.0,18,4,2015,5,2,0.0


In [90]:
categorical_cols=['payment_type','vendor_id','new_user']
other_df = pd.get_dummies(train[categorical_cols])
print other_df.columns
train
# other_df_test = pd.get_dummies(test[cat])

Index([u'payment_type_CRD', u'payment_type_CSH', u'payment_type_DIS',
       u'payment_type_NOC', u'payment_type_UNK', u'vendor_id_DST000401',
       u'vendor_id_DST000481', u'vendor_id_DST000532', u'vendor_id_DST000543',
       u'new_user_NO', u'new_user_YES'],
      dtype='object')


In [ ]:
useless_cols=['TID','store_and_fwd_flag','fare_amount','dropoff_latitude','dropoff_longitude',]

In [ ]:
fmt='%Y-%m-%d %H:%M:%S'
a=[1,2,3,4,5]
b=['a','b','c','d','e']
for x in np.arange(6):
    print x,'\n'

In [ ]:
useless_cols=['TID','store_and_fwd_flag','fare_amount']